# Задание: 
Для каждого кадра с камеры 1 нужно найти соответствующий (ближайший по времени) кадр с камеры 2. 
* Камеры записывают в формате (номер кадра, время его захвата) ~ (frame, timestamp).
* Камеры могут делать запись с разной частотой кадров в секунду и наличием шума (несоответствии времени кадра ожидаемому времени).

## Настройки/Гиперпараметры/Импорты

In [1]:
import time # для подсчёта времени работы
import numpy as np # для быстрой работы с массивами
from multiprocessing import Pool # для параллельного запуска (нужны процессы, а не потоки из multithreading/concurrent.futures)
from numba import njit # импорт компилятора для ускорения питона
from numba import float64, int32 # для создания сигнатур (почему-то крашится, поэтому в коде использование закоменчено)

In [2]:
cores_number = 2 # число ядер, на которые будет распараллеливаться выполнение кода
runs = 5 # число запусков для усреднения времени

## Вспомогательные функции

In [3]:
def make_timestamps(fps: int, st_ts: float, fn_ts: float) -> np.ndarray: # функция генерации записей с камеры
    """
    Создаёт np.array timestamps (массив времени записи кадра). 
    Этот массив дискретизирован по fps, но не равномерно.
    Возвращаемые timestamps отсортированы и уникальны.\n
    Parameters:
        * fps: среднее число кадров в секунду
        * st_ts: первый timestamp в последовательности (время первого кадра)
        * fn_ts: первый timestamp в последовательности (время последнего кадра) \n
    Returns:
        * np.ndarray: сгенерированные timestamps (массивы времени записи кадра)
    """
    timestamps = np.linspace(st_ts, fn_ts, int((fn_ts - st_ts) * fps)) # создаём временные записи из равномерного распределения
    timestamps += np.random.randn(len(timestamps)) # зашумляем кадры
    timestamps = np.unique(np.sort(timestamps)) # сортируем кадры по времени и оставляем только уникальные
    return timestamps

In [4]:
def calc_average_time(alg, data, runs: int=100) -> (float, np.ndarray): # функция для подсчёта среднего времени работы алгоритма
    """
    Функция для подсчёта среднего времени работы алгоритма.\n
    Parameters:
        * alg: алгоритм для тестирования
        * data: данные на вход алгоритма
        * runs: число запусков для усреднения времени
    Returns:
        * float: среднее время работы
        * np.ndarray: получившийся ответ
    """
    time_start = time.perf_counter() # замеряем время начала
    for i in range(runs): # делаем runs запусков
        answer = alg(*data) # вызываем алгоритм (*data для развёртывания переданных данных для алгоритма, если они в list-like формате)
        print(i, (time.perf_counter() - time_start)/(i+1)) # вывод среднего времени i-ой итерации (i+1 из-за нумерации с нуля)
    time_average = (time.perf_counter() - time_start) / runs # считаем среднее время
    return time_average, answer # возвращаем среднее время и получившийся ответ

In [5]:
# generate timestamps for the first camera
# timestamps1 = make_timestamps(30, time.time() - 100, time.time() + 3600 * 2)
# generate timestamps for the second camera
# timestamps2 = make_timestamps(60, time.time() + 200, time.time() + 3600 * 2.5)

timestamps1 = make_timestamps(30, time.time() - 100, time.time() + 900 * 2) # не 3600 (60 секунд * 60 минут), а лишь (15 * 60) = 15 минут
timestamps2 = make_timestamps(60, time.time() + 200, time.time() + 900 * 2.5) # не 3600 (60 секунд * 60 минут), а лишь (15 * 60) = 15 минут

In [6]:
timestamps1.shape

(57000,)

## Варианты решения

### 1) Глупо считаем разницу во времени между i-м кадром с первой камеры и всеми кадрами со второй камеры. В ответ записываем номер кадра со второй камеры с наименьшим модулем разности времени.

In [7]:
@njit #====================== использовать ли numba компилятор для функции ============================
def match_timestamps_naive(timestamps1: np.ndarray, timestamps2: np.ndarray) -> np.ndarray: 
    """
    * Делаем для каждого элемента первого массива: - O(n)
            * из всех элементов второго массива вычитается элемент первого массива (считаем delta) за O(2n) ~ O(n)
            * в получившихся delta ищем наименьший элемент за O(2n) ~ O(n)\n
    Итоговая сложность: O(размер_первого_массива * (размер_второго_массива + размер_второго_массива)) = O(n*(2n+2n)) = O(4n^2) ~ O(n^2)
    Parameters:
    * timestamps1: массив с временами с камеры 1
    * timestamps2: массив с временами с камеры 2\n
    Returns:
    * np.ndarray: массив соответствий кадров с камеры 1 к кадрам камеры 2
    """
    
    # так как в задании первая камера пишет в ~30 fps -> размер первого массива n; вторая камера пишет в ~60 fps -> размер второго массива не сильно больше 2n; сокращаем на константы и получаем n
    frames_count = timestamps1.shape[0] # число кадров на камере 1
    correspondence = np.zeros(frames_count, dtype=np.int32) # создаём массив под номера кадров с типом int32 для уменьшения потребляемой памяти

    #===================== вариант с циклом в np.arange ================================================
    # for frame in np.arange(frames_count): # идём по номерам кадров с первой камеры
    #--------------------- вариант с циклом в enumerate ------------------------------------------------
    for frame, frame_time in enumerate(timestamps1): # идём по кадрам
    #===================================================================================================
        #===================== вариант с сохранением delta =============================================
        # deltas = np.absolute(timestamps2 - timestamps1[frame]) # считаем разницу между временами
        # correspondence[frame] = deltas.argmin() # берём номер кадра с наименьшей разницей
        #--------------------- вариант без сохранения delta --------------------------------------------
        correspondence[frame] = np.absolute(timestamps2 - frame_time).argmin() # цикл с enumerate
        # correspondence[frame] = np.absolute(timestamps2 - timestamps1[frame]).argmin() # цикл с np.arange
        #===============================================================================================

    return correspondence

In [8]:
true_time, true_answer = calc_average_time(match_timestamps_naive, (timestamps1, timestamps2), runs=runs) # njit, цикл в enumerate

0 16.7497279
1 16.024130800000002
2 15.95187336666667
3 15.859213975
4 15.83067484


<pre>
- naive no njit + np.arange ~ 20.0 - 70.0 секунд
- naive no njit + enumerate ~ 18.0 - 19.0 секунд
- naive njit    + np.arange ~ 15.5 - 15.8 секунд
- naive njit    + enumerate ~ 15.0 - 15.7 секунд
</pre>

### 2) Параллельная версия первого алгоритма

In [9]:
from funcs import match_timestamps_naive # импорт функции, так как в notebook multiprocessing навсегда зависает при вызове функции из этого же notebook (???)

In [10]:
def match_timestamps_naive_parallel(timestamps1: np.ndarray, timestamps2: np.ndarray, func, cores_number: np.int8) -> np.ndarray: 
    """
    * Делаем для каждого элемента первого массива с распараллеливанием: - O(n/cores_number)
            * из всех элементов второго массива вычитается элемент первого массива (считаем delta) за O(2n) ~ O(n)
            * в получившихся delta ищем наименьший элемент за O(2n) ~ O(n)\n
    Итоговая сложность: O(размер_первого_массива / число_ядер * (размер_второго_массива + размер_второго_массива)) = O(n/cores_number*(2n+2n)) = O(4/cores_number*n^2) ~ O(n^2/cores_number)\n
    Parameters:
    * timestamps1: массив с временами с камеры 1
    * timestamps2: массив с временами с камеры 2
    * func: запускаемая в параллель функция
    * cores_number: число ядер, на которое будет распараллеливание\n
    Returns:
    * np.ndarray: массив соответствий кадров с камеры 1 к кадрам камеры 2
    """

    # так как в задании первая камера пишет в ~30 fps -> размер первого массива n; вторая камера пишет в ~60 fps -> размер второго массива 2n; сокращаем на константы и получаем n
    frames_count = timestamps1.shape[0] # число кадров с первой камеры
    with Pool(cores_number) as p: # запускаем cores_number процессов
        correspondence = np.concatenate(p.starmap(func, [(timestamps1[int(i*frames_count/cores_number):int((i+1)*frames_count/cores_number)], timestamps2) for i in np.arange(cores_number)]), dtype=np.int32)
        #  - каждому процессу отправляем равную часть кадров с первой камеры и все кадры со второй
        #  - конкатенируем вернувшиеся из процессов массивы с индексами
        # процесс вызывает переданную функцию func с параметрами: срез кадров с первой камеры, все кадры второй камеры
        # не обязательно передавать chunksize, так как распараллеливание уже идёт по кейсам/инпутам для функций
        # dtype=np.int32 для уменьшения потребляемой памяти и ускорения работы

    return correspondence

In [11]:
time_naive_parallel, answer_naive_parallel = calc_average_time(match_timestamps_naive_parallel, (timestamps1, timestamps2, match_timestamps_naive, cores_number), runs=runs) # подсчёт среднего времени (с njit и enumerate циклом)

0 10.9238766
1 11.08075015
2 10.947322066666672
3 10.957721324999998
4 10.934612280000001


In [12]:
np.all(answer_naive_parallel==true_answer) # проверка, что параллельная функция отработала корректно

True

<pre>
- parallel naive no njit + np.arange ~ 40.0 - 41.0 секунд
- parallel naive no njit + enumerate ~ 41.0 - 43.0 секунд
- parallel naive njit    + np.arange ~ 11.2 - 12.0 секунд
- parallel naive njit    + enumerate ~ 10.0 - 11.2 секунд
</pre>

### 3) Идём по последовательности кадров с первой камеры, запоминанием найденное соответствие со второй (номер delta, после которого разница во времени начинают увеличиваться). На следующем шаге по кадрам на второй камере начинаем идти с найденного соответствия на предыдущем шаге (так как кадры отсортированы в порядке неубывания - нет смысла рассматривать кадры, что заведомо имеют большую delta из-за увеличения времени на первой камере).

In [13]:
# @njit(float64[:](float64[:], float64[:], int32)) # использовать ли numba компилятор для функции с заданием сигнатуры для изначальной компиляции
@njit #====================== использовать ли numba компилятор для функции ============================
def match_timestamps_iterated(timestamps1: np.ndarray, timestamps2: np.ndarray, cam2_frame: np.int32=0) -> np.ndarray: 
    """
    * Ищем начальный кадр для второй камеры - O(2n), однако реальная сложность меньше, так как cam2_frame уже найден эвристически с учётом числа кадров на первой и второй камере
    * Делаем для каждого элемента (времени) с первой камеры: - O(n)
            * Вычитаем из соответствующего времени кадра второй камеры значение времени кадра с первой камеры - O(2n) в худшем случае, O(2) ~ O(1) в среднем, так как кадры распределены равномерно и во втором массиве не на порядок больше элементов
                    * если дельта уменьшилась - переходим к следующему кадру на второй камере и так далее, пока дельта уменьшается\n
                    * если дельта не уменьшилась - возвращаем предыдущий кадр, как самый близкий по времени\n
    Итоговая сложность: O(размер_второго_массива + размер_первого_массива * размер_второго_массива) ~ [в среднем] ~ O(размер_второго_массива + размер_первого_массива * 1) ~ O(n)\n
    Parameters:
    * timestamps1: массив с временами с камеры 1
    * timestamps2: массив с временами с камеры 2
    * cam2_frame: с какого элемента рассматривать кадры с камеры 2 (важен при параллельном запуске)\n
    Returns:
    * np.ndarray: массив соответствий кадров с камеры 1 к кадрам камеры 2
    """
    
    # так как в задании первая камера пишет в ~30 fps -> размер первого массива n; вторая камера пишет в ~60 fps -> размер второго массива не сильно больше 2n; сокращаем на константы и получаем n
    frames_count = timestamps1.shape[0] # число кадров на камере 1
    max_frame = timestamps2.shape[0] - 1 # максимальный номер кадра, что может соответствовать кадру с первой камеры (-1 из-за индексации с нуля)
    correspondence = np.zeros(frames_count, dtype=np.int32) # создаём массив под номера кадров с типом int32 для уменьшения потребляемой памяти

    if 0 > cam2_frame or cam2_frame > max_frame: # защита от дурака с cam2_frame
        cam2_frame = 0 # зануление начального кадра второй камеры

    # подбираем кадр на второй камере, с которого будем начинать (эвристически уже передали ожидаемый cam2_frame, нужно его лишь правильно сместить, если он "обогнал" кадры с камеры 1)
    delta = np.abs(timestamps2[cam2_frame] - timestamps1[0]) # текущая разность времени
    while np.abs(timestamps2[max(0, cam2_frame-1)] - timestamps1[0]) < delta: # если дельта уменьшается при предыдущем кадре, то переходим на него (условие — строго меньше, так как времена всех кадров уникальны)
        cam2_frame -= 1 # переходим на предыдущий кадр (без max, так как при нуле мы бы сюда не зашли в цикл)
        delta = np.abs(timestamps2[cam2_frame] - timestamps1[0]) # обновляем текущее значение delta (без max, так как при нуле мы бы не зашли в цикл)
    # после этого цикла мы либо находимся на оптимальном соответствии первого кадра первой камеры с кадром со второй камеры, либо оптимум — далее по времени (но не раньше!)

    #===================== вариант с циклом в np.arange ================================================
    # for frame in np.arange(frames_count): # идём по номерам кадров с первой камеры
    #     delta = np.abs(timestamps2[cam2_frame] - timestamps1[frame]) # текущая разность времени
    #     while np.abs(timestamps2[min(cam2_frame+1, max_frame)] - timestamps1[frame]) < delta: # если при переходе на следующий кадр дельта (разница времени) уменьшилась, то обновляем delta и соответствующий кадр на второй камере 
    #         cam2_frame += 1 # переходим на следующий кадр
    #         delta = np.abs(timestamps2[cam2_frame] - timestamps1[frame])  # обновляем delta
    #     correspondence[frame] = cam2_frame # если дельта перестала уменьшаеться — записываем найденный кадр
    #--------------------- вариант с циклом в enumerate ------------------------------------------------
    for frame, frame_time in enumerate(timestamps1): # идём по кадрам
        delta = np.abs(timestamps2[cam2_frame] - frame_time) # текущая разность времени
        while np.abs(timestamps2[min(cam2_frame+1, max_frame)] - frame_time) < delta: # если при переходе на следующий кадр дельта (разница времени) уменьшилась, то обновляем delta и соответствующий кадр на второй камере 
            cam2_frame += 1 # переходим на следующий кадр
            delta = np.abs(timestamps2[cam2_frame] - frame_time)  # обновляем delta
        correspondence[frame] = cam2_frame # если дельта перестала уменьшаеться — записываем найденный кадр
    #===============================================================================================

    return correspondence

In [14]:
time_iterated, answer_iterated = calc_average_time(match_timestamps_iterated, (timestamps1, timestamps2), runs=runs) # подсчёт среднего времени (с njit и enumerate циклом)
# время сильно разнится из-за создания бинаря для функции с njit при первом запуске (последующие запуски идут всегда быстрее)

0 0.345716699999997
1 0.1736082500000009
2 0.11603346666666425
3 0.08725662499999487
4 0.07004000000000019


In [15]:
np.all(answer_iterated==true_answer) # проверка, что функция отработала корректно

True

<pre>
- iterated no njit + np.arange ~ 0.7451 секунд
- iterated no njit + enumerate ~ 0.6525 секунд
- iterated njit    + np.arange ~ 0.0882 секунд
- iterated njit    + enumerate ~ 0.0795 секунд
</pre>

### 4) Параллельная реализация третьего алгоритма

In [16]:
from funcs import match_timestamps_iterated # импорт функции, так как в notebook multiprocessing навсегда зависает при вызове функции из этого же notebook (???)

In [21]:
def match_timestamps_iterated_parallel(timestamps1: np.ndarray, timestamps2: np.ndarray, func, cores_number: np.int8) -> np.ndarray: 
    """
    * Ищем начальный кадр для второй камеры - O(2n), однако реальная сложность меньше, так как cam2_frame уже найден эвристически с учётом числа кадров на первой и второй камере
    * Делаем для каждого элемента первого массива (времени) с распараллеливанием: - O(n/cores_number)
            * Вычитаем из соответствующего времени кадра второй камеры значение времени кадра с первой камеры - O(2n) в худшем случае, O(2) ~ O(1) в среднем, так как кадры распределены равномерно и во втором массиве не на порядок больше элементов
                    * если дельта уменьшилась - переходим к следующему кадру на второй камере и так далее, пока дельта уменьшается
                    * если дельта не уменьшилась - возвращаем предыдущий кадр, как самый близкий по времени\n
    Итоговая сложность: O(размер_второго_массива + размер_первого_массива / число_ядер * размер_второго_массива) ~ [в среднем] ~ O(размер_второго_массива + размер_первого_массива / число_ядер * 1) ~ O(n/cores_number)\n
    Parameters:
    * timestamps1: массив с временами с камеры 1
    * timestamps2: массив с временами с камеры 2
    * func: запускаемая в параллель функция
    * cores_number: число ядер, на которое будет распараллеливание\n
    Returns:
    * np.ndarray: массив соответствий кадров с камеры 1 к кадрам камеры 2
    """

    # так как в задании первая камера пишет в ~30 fps -> размер первого массива n; вторая камера пишет в ~60 fps -> размер второго массива 2n; сокращаем на константы и получаем n
    frames_count = timestamps1.shape[0] # число кадров с первой камеры
    with Pool(cores_number) as p: # запускаем cores_number процессов
        correspondence = np.concatenate(p.starmap(func, [(timestamps1[int(i*frames_count/cores_number):int((i+1)*frames_count/cores_number)], timestamps2, int(2*i*frames_count/cores_number)) for i in np.arange(cores_number)]), dtype=np.int32)
        #  - каждому процессу отправляем равную часть кадров с первой камеры и все кадры со второй, а также ожидаемый номер соответствия первого кадра на камере 1 с камерой 2
        #  - конкатенируем вернувшиеся из процессов массивы с индексами
        # процесс вызывает переданную функцию func с параметрами: срез кадров с первой камеры, все кадры второй камеры, а также ожидаемый номер соответствия первого кадра на камере 1 с камерой 2
        # не обязательно передавать chunksize, так как распараллеливание уже идёт по кейсам/инпутам для функций
        # dtype=np.int32 для уменьшения потребляемой памяти и ускорения работы

    return correspondence

In [22]:
time_iterated_parallel, answer_iterated_parallel = calc_average_time(match_timestamps_iterated_parallel, (timestamps1, timestamps2, match_timestamps_iterated, cores_number), runs=runs) # подсчёт среднего времени (с njit и enumerate циклом)

0 2.3995212999999467
1 2.217156999999986
2 2.1871334999999967
3 2.172527725000009
4 2.1376731999999947


In [23]:
np.all(answer_iterated_parallel==true_answer) # проверка, что функция отработала корректно

True

<pre>
- parallel iterated no njit + np.arange ~ 1.3736 секунд
- parallel iterated no njit + enumerate ~ 1.3057 секунд
- parallel iterated njit    + np.arange ~ 2.4365 секунд
- parallel iterated njit    + enumerate ~ 2.1035 секунд
</pre>

# Выводы

* Лучшим вариантом, как и ожидалось, оказался итерационный алгоритм (njit + enumerate + no parallel).
* Циклы с enumerate показали себя лучше, чем циклы с np.arange.
* Параллелизация может ускорить выполнение кода, однако её эффективность зависит от изначальной времени работы (возможен случай, что на распараллеливание уходит больше времени, чем на сам подсчёт).
* njit ускоряет код вплоть x10, так как компилирует бинарные файлы для функций (без сигнатуры бинарный файл создастся при первом вызове, а не при определении функции), однако иногда он может наоборот — замедлить (см вариант 4 с параллельным запуском).